In [ ]:
import dolfinx.mesh
import mpi4py.MPI

In [ ]:
import viskex

Generate and plot meshes

In [ ]:
interval = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 5)
square_tria = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, 4, 4, dolfinx.mesh.CellType.triangle)
square_quad = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, 4, 4, dolfinx.mesh.CellType.quadrilateral)
cube_tetra = dolfinx.mesh.create_unit_cube(mpi4py.MPI.COMM_WORLD, 3, 3, 3, dolfinx.mesh.CellType.tetrahedron)
cube_hexa = dolfinx.mesh.create_unit_cube(mpi4py.MPI.COMM_WORLD, 3, 3, 3, dolfinx.mesh.CellType.hexahedron)

In [ ]:
viskex.plot_mesh(interval)

In [ ]:
viskex.plot_mesh(square_tria)

In [ ]:
viskex.plot_mesh(square_quad)

In [ ]:
viskex.plot_mesh(cube_tetra)

In [ ]:
viskex.plot_mesh(cube_hexa)